# The diet problem

[Tutorial](https://nbviewer.jupyter.org/github/Pyomo/PyomoGallery/blob/master/diet/DietProblem.ipynb)

In [1]:
from pyomo import environ

infinity = float("inf")

## Problem

Satisfy the daily nutrient requiremennts at a minimum cost.

### Sets

- $F$: set of foods
- $N$: set of nutrients

### Parameters

- $c_i$: cost per serving of food $i$, $\forall i \in F$
- $a_{ij}$: cost of nutrient $j$ in food $i$, $\forall i \in F$, $\forall j \in N$
- $Nmin_j$, $Nmax_j$: minmum and maximum level of nutrient $j$, $\forall j \in N$
- $V_i$: volume per serving of food $j$, $\forall j \in F$
- $Vmax$: maximum volume of food consumed

### Variables

- $x_i$: number of servings of food $i$ to consume

### Objective

Minimize the cost of food: $\sum _{i \in F} c_i * x_i$

### Constraints

- limit the nutrient consumption for each nutriend $j \in N$: $Nmin_j \leq \sum_{i \in F} a_{ij} x_i \leq Nmax_j$, $\forall j \in N$
- limit the volume of food consumed: $\sum_{i \in F} V_i x_i \leq Vmax$
- lower bound on food consumption: $x_i \geq 0$, $\forall_i \in F$

## Coding the problem

In [2]:
model = environ.AbstractModel()

### Sets

In [3]:
model.F = environ.Set()  # Foods
model.N = environ.Set()  # Nutrients

### Parameters

In [4]:
model.c = environ.Param(model.F, within=environ.PositiveReals)
model.a = environ.Param(model.F, model.N, within=environ.NonNegativeReals)
model.Nmin = environ.Param(model.N, within=environ.NonNegativeReals, default=0.0)
model.Nmax = environ.Param(model.N, within=environ.NonNegativeReals, default=infinity)
model.V = environ.Param(model.F, within=environ.PositiveReals)
model.Vmax = environ.Param(within=environ.PositiveReals)

### Variables

In [5]:
model.x = environ.Var(model.F, within=environ.NonNegativeIntegers)

### Objective

In [6]:
def cost_rule(model):
    return sum(model.c[i] * model.x[i] for i in model.F)


model.cost = environ.Objective(rule=cost_rule)

### Constraints

In [7]:
def nutrient_rule(model, j):
    value = sum(model.a[i, j] * model.x[i] for i in model.F)
    return model.Nmin[j] <= value <= model.Nmax[j]


def volume_rule(model):
    return sum(model.V[i] * model.x[i] for i in model.F) <= model.Vmax


model.nutrient_limit = environ.Constraint(model.N, rule=nutrient_rule)
model.volume = environ.Constraint(rule=volume_rule)

## Data

In [8]:
!cat diet_data.dat

param:  F:                          c     V  :=
  "Cheeseburger"                 1.84   4.0  
  "Ham Sandwich"                 2.19   7.5  
  "Hamburger"                    1.84   3.5  
  "Fish Sandwich"                1.44   5.0  
  "Chicken Sandwich"             2.29   7.3  
  "Fries"                         .77   2.6  
  "Sausage Biscuit"              1.29   4.1  
  "Lowfat Milk"                   .60   8.0 
  "Orange Juice"                  .72  12.0 ;

param Vmax := 75.0;

param:  N:       Nmin   Nmax :=
        Cal      2000      .
        Carbo     350    375
        Protein    55      .
        VitA      100      .
        VitC      100      .
        Calc      100      .
        Iron      100      . ;

param a:
                               Cal  Carbo Protein   VitA   VitC  Calc  Iron :=
  "Cheeseburger"               510     34     28     15      6    30    20
  "Ham Sandwich"               370     35     24     15     10    20    20
  "Hamburger"                  500     42